In [1]:
!pip install pandas
!pip install ipython-sql

import csv, sqlite3

con = sqlite3.connect("FinalDB.db")
cur = con.cursor()


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%load_ext sql

In [3]:
import pandas as pd

#read csv file from the given url into pandas dataframes
census_data_df = pd.read_csv('ChicagoCensusData.csv')

#convert csv file to a table in sqlite with the csv data loaded in it
census_data_df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")


78

In [4]:
chicago_public_schools_df = pd.read_csv('ChicagoPublicSchools.csv')

chicago_public_schools_df.to_sql(name="CHICAGO_PUBLIC_SCHOOLS", con= con, if_exists='replace', index=False)

566

In [5]:
chicago_crime_data_df = pd.read_csv('ChicagoCrimeData.csv')

chicago_crime_data_df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False,method="multi")

533

In [6]:
%sql sqlite:///FinalDB.db


In [7]:
%sql select name from  sqlite_master where type='table';


 * sqlite:///FinalDB.db
Done.


name
CENSUS_DATA
CHICAGO_PUBLIC_SCHOOLS
CHICAGO_CRIME_DATA


## Problems

Now write and execute SQL queries to solve assignment problems

### Problem 1

##### Find the total number of crimes recorded in the CRIME table.

In [8]:
%sql select count(*) AS total_crimes from CHICAGO_CRIME_DATA;

 * sqlite:///FinalDB.db
Done.


total_crimes
533


### Problem 2

##### List community areas with per capita income less than 11000.

In [9]:
%sql select "COMMUNITY_AREA_NAME", "PER_CAPITA_INCOME" from CENSUS_DATA where "PER_CAPITA_INCOME" < 11000 order by "PER_CAPITA_INCOME";

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
Riverdale,8201
South Lawndale,10402
Fuller Park,10432
West Garfield Park,10934


### Problem 3

##### List all case numbers for crimes  involving minors?(children are not considered minors for the purposes of crime analysis)

In [10]:
%sql select CASE_NUMBER, DESCRIPTION from CHICAGO_CRIME_DATA where DESCRIPTION like '%minor%';

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,DESCRIPTION
HL266884,SELL/GIVE/DEL LIQUOR TO MINOR
HK238408,ILLEGAL CONSUMPTION BY MINOR


### Problem 4

##### List all kidnapping crimes involving a child?

In [11]:
%sql select PRIMARY_TYPE, DESCRIPTION from CHICAGO_CRIME_DATA where PRIMARY_TYPE='KIDNAPPING' and DESCRIPTION like '%child%';

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE,DESCRIPTION
KIDNAPPING,CHILD ABDUCTION/STRANGER


### Problem 5

##### What kinds of crimes were recorded at schools?


In [12]:
%sql select distinct PRIMARY_TYPE, LOCATION_DESCRIPTION from CHICAGO_CRIME_DATA where LOCATION_DESCRIPTION like '%school%';


 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE,LOCATION_DESCRIPTION
BATTERY,"SCHOOL, PUBLIC, GROUNDS"
BATTERY,"SCHOOL, PUBLIC, BUILDING"
CRIMINAL DAMAGE,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, BUILDING"
ASSAULT,"SCHOOL, PUBLIC, GROUNDS"
CRIMINAL TRESPASS,"SCHOOL, PUBLIC, GROUNDS"
PUBLIC PEACE VIOLATION,"SCHOOL, PRIVATE, BUILDING"
PUBLIC PEACE VIOLATION,"SCHOOL, PUBLIC, BUILDING"


### Problem 6

##### List the average safety score of school.

In [13]:
%sql SELECT AVG("Safety_Score") AS "AverageSafetyScore" FROM CHICAGO_PUBLIC_SCHOOLS;


 * sqlite:///FinalDB.db
Done.


AverageSafetyScore
49.50487329434698


### Problem 7

##### List 5 community areas with highest % of households below poverty line

In [14]:
%sql select "COMMUNITY_AREA_NAME", "PERCENT_HOUSEHOLDS_BELOW_POVERTY" from CENSUS_DATA order by "PERCENT_HOUSEHOLDS_BELOW_POVERTY" desc nulls last limit 5;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


### Problem 8

##### Which community area is most crime prone?

In [15]:
%sql select WARD, COMMUNITY_AREA_NUMBER, count(COMMUNITY_AREA_NUMBER) from CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC LIMIT 1;

 * sqlite:///FinalDB.db
Done.


WARD,COMMUNITY_AREA_NUMBER,count(COMMUNITY_AREA_NUMBER)
29.0,25.0,43


### Problem 9

##### Use a sub-query to find the name of the community area with highest hardship index

In [16]:
%sql select "COMMUNITY_AREA_NAME", "HARDSHIP_INDEX" from CENSUS_DATA \
   where "COMMUNITY_AREA_NAME" in \
   ( select "COMMUNITY_AREA_NAME" from CENSUS_DATA order by "HARDSHIP_INDEX" desc limit 1 );

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,HARDSHIP_INDEX
Riverdale,98.0


### Problem 10

##### Use a sub-query to determine the Community Area Name with most number of crimes?

In [17]:
%sql select "COMMUNITY_AREA_NAME" from CENSUS_DATA \
   where "COMMUNITY_AREA_NUMBER" in \
   (select count(COMMUNITY_AREA_NUMBER) from CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC LIMIT 1);

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
South Shore
